# Importing important libraries


In [31]:
import pandas as pd
import matplotlib.pyplot as plt
import math

# Reading of a CSV file and extraction of itss important data 

In this section we are reading a CSV file which come from the official NBA webside. The CSV file is In the Github under the name "ranking.csv". 
Then we create a Data frame which contains the teams ranking during the last day of the different NBA seasons which is the second of april.
As an example we can see that the Phoenix were leading the West-NBA conference before the playoffs of 2022

In [49]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv("C:/Users/cestm/Downloads/statistics/ranking.csv")

# Extract specific columns 'TEAM_ID', 'STANDINGSDATE', 'TEAM' and filter for 'CONFERENCE'=='West'
selected_columns = df[['TEAM_ID', 'STANDINGSDATE', 'TEAM', 'CONFERENCE']][df['CONFERENCE'] == 'West']

# Filter rows where 'STANDINGSDATE' ends with "-12-22"
filtered_rows = selected_columns[selected_columns['STANDINGSDATE'].str.endswith('-04-02')]

# Display the filtered DataFrame
print(filtered_rows)




           TEAM_ID STANDINGSDATE          TEAM CONFERENCE
3960    1610612756    2022-04-02       Phoenix       West
3961    1610612763    2022-04-02       Memphis       West
3962    1610612744    2022-04-02  Golden State       West
3963    1610612742    2022-04-02        Dallas       West
3964    1610612743    2022-04-02        Denver       West
...            ...           ...           ...        ...
118537  1610612762    2015-04-02          Utah       West
118538  1610612743    2015-04-02        Denver       West
118539  1610612758    2015-04-02    Sacramento       West
118540  1610612747    2015-04-02   L.A. Lakers       West
118541  1610612750    2015-04-02     Minnesota       West

[284 rows x 4 columns]


# Making a dataframe with only the rankings

As our only criteria for our elo based model is the rankings during the different seasons. We are here making a new dataframe with only the rankings of the different West-NBA conference seasons.

In [33]:
rankingyear2022 = filtered_rows[filtered_rows['STANDINGSDATE'].str.startswith('2022-')]
rankingyear2021 = filtered_rows[filtered_rows['STANDINGSDATE'].str.startswith('2021-')]
rankingyear2020 = filtered_rows[filtered_rows['STANDINGSDATE'].str.startswith('2020-')]
rankingyear2019 = filtered_rows[filtered_rows['STANDINGSDATE'].str.startswith('2019-')]
rankingyear2018 = filtered_rows[filtered_rows['STANDINGSDATE'].str.startswith('2018-')]
rankingyear2017 = filtered_rows[filtered_rows['STANDINGSDATE'].str.startswith('2017-')]
rankingyear2016 = filtered_rows[filtered_rows['STANDINGSDATE'].str.startswith('2016-')]
rankingyear2015 = filtered_rows[filtered_rows['STANDINGSDATE'].str.startswith('2015-')]
rankingyear2014 = filtered_rows[filtered_rows['STANDINGSDATE'].str.startswith('2014-')]

# Assuming you have already defined rankingyear2022, rankingyear2021, ..., rankingyear2014

# Concatenate the DataFrames along the rows
dff = pd.concat([rankingyear2022, rankingyear2021, rankingyear2020, rankingyear2019, rankingyear2018, rankingyear2017, rankingyear2016, rankingyear2015, rankingyear2014])

# Display the concatenated DataFrame
print(dff)


          TEAM_ID STANDINGSDATE          TEAM CONFERENCE
3960   1610612756    2022-04-02       Phoenix       West
3961   1610612763    2022-04-02       Memphis       West
3962   1610612744    2022-04-02  Golden State       West
3963   1610612742    2022-04-02        Dallas       West
3964   1610612743    2022-04-02        Denver       West
...           ...           ...           ...        ...
35784  1610612743    2014-04-02        Denver       West
35785  1610612740    2014-04-02   New Orleans       West
35786  1610612758    2014-04-02    Sacramento       West
35787  1610612747    2014-04-02   L.A. Lakers       West
35788  1610612762    2014-04-02          Utah       West

[135 rows x 4 columns]


# Making a CSV file of that new dataframe

In [34]:

dff.to_csv("CLASSEMENT")


# Elo Rating System Based on Team Rankings

This script calculates Elo ratings for teams based on their rankings in each season, with an adjustment at the start of each season where the new Elo rating is a weighted average of 85% of the previous season's Elo rating and 15% of the initial Elo rating of 1500.

## Steps of the Algorithm

1. **Initialization of Elo Ratings**: All teams start with an initial Elo rating of 1500.
2. **Definition of Elo Rating Update Function**: This function updates Elo ratings based on match outcomes.
3. **Simulation of Matches and Elo Rating Updates**: Teams are compared based on their rankings, and their Elo ratings are updated accordingly.
4. **Adjustment of Elo Ratings at the Start of Each Season**: Calculate the new Elo rating at the beginning of each season as 85% of the previous season's Elo rating plus 15% of 1500.




In [53]:
import pandas as pd

# Lire le fichier CSV dans un DataFrame
df = pd.read_csv("CLASSEMENTT.csv")

# Initialisation des cotes Elo
initial_elo = 1500
elo_ratings = {team: initial_elo for team in df['TEAM'].unique()}

# Définir la fonction de mise à jour des cotes Elo
def update_elo(winner_elo, loser_elo, K=32):
    expected_win_winner = 1 / (1 + 10 ** ((loser_elo - winner_elo) / 400))
    expected_win_loser = 1 / (1 + 10 ** ((winner_elo - loser_elo) / 400))
    
    new_winner_elo = winner_elo + K * (1 - expected_win_winner)
    new_loser_elo = loser_elo + K * (0 - expected_win_loser)
    
    return new_winner_elo, new_loser_elo

# Fonction pour mettre à jour les cotes Elo au début de chaque saison
def adjust_elo_start_of_season(elo_ratings):
    for team in elo_ratings:
        elo_ratings[team] = 0.85 * elo_ratings[team] + 0.15 * initial_elo

# Obtenir les années uniques dans le dataset
df['YEAR'] = pd.to_datetime(df['STANDINGSDATE']).dt.year
unique_years = sorted(df['YEAR'].unique())

# Simuler les matchs et mettre à jour les cotes Elo pour chaque saison
for year in unique_years:
    # Filtrer les données pour l'année en cours
    yearly_data = df[df['YEAR'] == year].sort_values(by='TEAM_ID')
    teams = yearly_data['TEAM'].values
    
    # Comparer chaque paire d'équipes et mettre à jour les cotes Elo
    for i in range(len(teams)):
        for j in range(i + 1, len(teams)):
            team1 = teams[i]
            team2 = teams[j]
            
            team1_elo = elo_ratings[team1]
            team2_elo = elo_ratings[team2]
            
            if i < j:  # Supposons que l'équipe i a gagné contre l'équipe j (parce que i est mieux classée que j)
                new_team1_elo, new_team2_elo = update_elo(team1_elo, team2_elo)
            else:  # Supposons que l'équipe j a gagné contre l'équipe i
                new_team2_elo, new_team1_elo = update_elo(team2_elo, team1_elo)
                
            elo_ratings[team1] = new_team1_elo
            elo_ratings[team2] = new_team2_elo
    
    # Ajuster les cotes Elo au début de la saison suivante
    if year != unique_years[-1]:  # Pas besoin de mettre à jour après la dernière année
        adjust_elo_start_of_season(elo_ratings)

# Créer un DataFrame pour les cotes Elo
elo_df = pd.DataFrame(elo_ratings.items(), columns=['TEAM', 'ELO'])

# Trier le DataFrame par cotes Elo décroissantes
sorted_df = elo_df.sort_values(by='ELO', ascending=False)

# Afficher le DataFrame trié
print(sorted_df)


             TEAM          ELO
2     New Orleans  1889.239387
8          Dallas  1826.140681
0          Denver  1767.948363
10   Golden State  1712.992794
14        Houston  1660.092709
4     LA Clippers  1608.192630
12    L.A. Lakers  1557.114032
15  L.A. Clippers  1514.083117
9       Minnesota  1505.769467
3         Phoenix  1453.740408
7        Portland  1400.436150
5      Sacramento  1345.200296
13    San Antonio  1287.263093
11  Oklahoma City  1225.689762
6            Utah  1159.327892
1         Memphis  1086.769219


In [54]:

def match_outcome_probability(team1, team2):
    # Access Elo ratings using .loc[] for label-based indexing
    elo_team1 = elo_df.loc[elo_df['TEAM'] == str(team1), 'ELO'].values[0]
    elo_team2 = elo_df.loc[elo_df['TEAM'] == str(team2), 'ELO'].values[0]

    delta_elo = elo_team1 - elo_team2
    probability_team1_win = 1 / (1 + math.pow(10, -delta_elo / 400))
    probability_team2_win = 1 / (1 + math.pow(10, delta_elo / 400))
    #probability_draw = 1 - probability_team1_win - probability_team2_win
    return print(f"{probability_team1_win, probability_team2_win}")

print(match_outcome_probability('Golden State', 'Minnesota'))


(0.7672545689160419, 0.23274543108395807)
None


In [38]:
import pandas as pd

# Charger les données
dfa = pd.read_csv("C:/Users/cestm/Downloads/statistics/ranking.csv")

# Extraire les colonnes spécifiques et filtrer pour 'CONFERENCE' == 'East'
selected_columns = dfa[['TEAM_ID', 'STANDINGSDATE', 'TEAM', 'CONFERENCE']]
filtered_columns = selected_columns[selected_columns['CONFERENCE'] == 'East']

# Filtrer les lignes où 'STANDINGSDATE' se termine par "-04-02"
filtered_rows = filtered_columns[filtered_columns['STANDINGSDATE'].str.endswith('-04-02')]

# Afficher les résultats filtrés
print(filtered_rows)


           TEAM_ID STANDINGSDATE          TEAM CONFERENCE
8235    1610612748    2022-04-02         Miami       East
8236    1610612749    2022-04-02     Milwaukee       East
8237    1610612738    2022-04-02        Boston       East
8238    1610612755    2022-04-02  Philadelphia       East
8239    1610612761    2022-04-02       Toronto       East
...            ...           ...           ...        ...
207142  1610612754    2015-04-02       Indiana       East
207143  1610612765    2015-04-02       Detroit       East
207144  1610612753    2015-04-02       Orlando       East
207145  1610612755    2015-04-02  Philadelphia       East
207146  1610612752    2015-04-02      New York       East

[285 rows x 4 columns]


In [55]:
rankingyear2022 = filtered_rowss[filtered_rowss['STANDINGSDATE'].str.startswith('2022-')]
rankingyear2021 = filtered_rowss[filtered_rowss['STANDINGSDATE'].str.startswith('2021-')]
rankingyear2020 = filtered_rowss[filtered_rowss['STANDINGSDATE'].str.startswith('2020-')]
rankingyear2019 = filtered_rowss[filtered_rowss['STANDINGSDATE'].str.startswith('2019-')]
rankingyear2018 = filtered_rowss[filtered_rowss['STANDINGSDATE'].str.startswith('2018-')]
rankingyear2017 = filtered_rowss[filtered_rowss['STANDINGSDATE'].str.startswith('2017-')]
rankingyear2016 = filtered_rowss[filtered_rowss['STANDINGSDATE'].str.startswith('2016-')]
rankingyear2015 = filtered_rowss[filtered_rowss['STANDINGSDATE'].str.startswith('2015-')]
rankingyear2014 = filtered_rowss[filtered_rowss['STANDINGSDATE'].str.startswith('2014-')]

# Assuming you have already defined rankingyear2022, rankingyear2021, ..., rankingyear2014

# Concatenate the DataFrames along the rows
dffa = pd.concat([rankingyear2022, rankingyear2021, rankingyear2020, rankingyear2019, rankingyear2018, rankingyear2017, rankingyear2016, rankingyear2015, rankingyear2014])

# Display the concatenated DataFrame
print(dffa)

           TEAM_ID STANDINGSDATE          TEAM CONFERENCE
8235    1610612748    2022-04-02         Miami       East
8236    1610612749    2022-04-02     Milwaukee       East
8237    1610612738    2022-04-02        Boston       East
8238    1610612755    2022-04-02  Philadelphia       East
8239    1610612761    2022-04-02       Toronto       East
...            ...           ...           ...        ...
124012  1610612765    2014-04-02       Detroit       East
124013  1610612738    2014-04-02        Boston       East
124014  1610612753    2014-04-02       Orlando       East
124015  1610612755    2014-04-02  Philadelphia       East
124016  1610612749    2014-04-02     Milwaukee       East

[135 rows x 4 columns]


In [56]:

dffa.to_csv("CLASSEMENTS")

In [58]:
import pandas as pd

# Lire le fichier CSV dans un DataFrame
df = pd.read_csv("CLASSEMENTS.csv")

# Initialisation des cotes Elo
initial_elo = 1500
elo_ratings = {team: initial_elo for team in df['TEAM'].unique()}

# Définir la fonction de mise à jour des cotes Elo
def update_elo(winner_elo, loser_elo, K=32):
    expected_win_winner = 1 / (1 + 10 ** ((loser_elo - winner_elo) / 400))
    expected_win_loser = 1 / (1 + 10 ** ((winner_elo - loser_elo) / 400))
    
    new_winner_elo = winner_elo + K * (1 - expected_win_winner)
    new_loser_elo = loser_elo + K * (0 - expected_win_loser)
    
    return new_winner_elo, new_loser_elo

# Fonction pour mettre à jour les cotes Elo au début de chaque saison
def adjust_elo_start_of_season(elo_ratings):
    for team in elo_ratings:
        elo_ratings[team] = 0.85 * elo_ratings[team] + 0.15 * initial_elo

# Obtenir les années uniques dans le dataset
df['YEAR'] = pd.to_datetime(df['STANDINGSDATE']).dt.year
unique_years = sorted(df['YEAR'].unique())

# Simuler les matchs et mettre à jour les cotes Elo pour chaque saison
for year in unique_years:
    # Filtrer les données pour l'année en cours
    yearly_data = df[df['YEAR'] == year].sort_values(by='TEAM_ID')
    teams = yearly_data['TEAM'].values
    
    # Comparer chaque paire d'équipes et mettre à jour les cotes Elo
    for i in range(len(teams)):
        for j in range(i + 1, len(teams)):
            team1 = teams[i]
            team2 = teams[j]
            
            team1_elo = elo_ratings[team1]
            team2_elo = elo_ratings[team2]
            
            if i < j:  # Supposons que l'équipe i a gagné contre l'équipe j (parce que i est mieux classée que j)
                new_team1_elo, new_team2_elo = update_elo(team1_elo, team2_elo)
            else:  # Supposons que l'équipe j a gagné contre l'équipe i
                new_team2_elo, new_team1_elo = update_elo(team2_elo, team1_elo)
                
            elo_ratings[team1] = new_team1_elo
            elo_ratings[team2] = new_team2_elo
    
    # Ajuster les cotes Elo au début de la saison suivante
    if year != unique_years[-1]:  # Pas besoin de mettre à jour après la dernière année
        adjust_elo_start_of_season(elo_ratings)

# Créer un DataFrame pour les cotes Elo
elo_df = pd.DataFrame(elo_ratings.items(), columns=['TEAM', 'ELO'])

# Trier le DataFrame par cotes Elo décroissantes
sorted_df = elo_df.sort_values(by='ELO', ascending=False)

# Afficher le DataFrame trié
print(sorted_df)


            TEAM          ELO
7        Atlanta  1890.172661
2         Boston  1827.087426
6      Cleveland  1768.900362
5        Chicago  1713.944453
0          Miami  1661.040049
1      Milwaukee  1609.314841
8       Brooklyn  1558.075763
11      New York  1506.722030
14       Orlando  1454.682931
12       Indiana  1401.367898
3   Philadelphia  1346.120409
4        Toronto  1288.170269
10    Washington  1226.581824
13       Detroit  1160.201199
9      Charlotte  1087.617885


In [59]:
def match_outcome_probability(team1, team2):
    # Access Elo ratings using .loc[] for label-based indexing
    elo_team1 = sorted_df.loc[sorted_df['TEAM'] == str(team1), 'ELO'].values[0]
    elo_team2 = sorted_df.loc[sorted_df['TEAM'] == str(team2), 'ELO'].values[0]

    delta_elo = elo_team1 - elo_team2
    probability_team1_win = 1 / (1 + math.pow(10, -delta_elo / 400))
    probability_team2_win = 1 / (1 + math.pow(10, delta_elo / 400))
    #probability_draw = 1 - probability_team1_win - probability_team2_win
    return print(f"{probability_team1_win, probability_team2_win}")

print(match_outcome_probability('Toronto', 'New York'))


(0.2213037130503924, 0.7786962869496076)
None
